In [1]:
from tqdm.notebook import tqdm
from IPython.display import display, HTML

import numpy as np
import pandas as pd

import plotly.graph_objects as go

import datetime
from pathlib import Path

import intake

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
import itertools

from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras import backend as K

# from tensorflow.keras.layers import ConvLSTM2D, Conv2D, Conv3D, Dense, TimeDistributed, MaxPooling2D 
# from tensorflow.keras.layers import Activation, Dropout, Flatten, BatchNormalization, Input
# from tensorflow.keras.layers import LSTM, GlobalAveragePooling3D, GlobalAveragePooling2D
# from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling3D
from tensorflow.keras.layers import *

2024-05-21 09:57:45.321905: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-21 09:57:46.212443: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
catalog = intake.open_catalog(Path('../catalog.yml'))
source = getattr(catalog, 'treesat_multi')
df = source.read()

In [3]:
selected_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']

In [4]:
target = source.metadata['categories']['multi'] # multi / trinary
labels = df[target].to_numpy()

model_dir = Path('models').joinpath('seasons')
model_dir.mkdir(parents=True, exist_ok=True)

data_dir = Path('seasonal_median')

In [5]:
def get_features(years, seasons, data_dir):
    selected_data = []
    for year in years:
        for season in seasons:
            filepath = data_dir.joinpath(f'processed_treesat_{year}{str(season).zfill(2)}.npy')
            with open(filepath, 'rb') as f:
                data = np.load(f)
            selected_data.append(data)
    return np.stack(selected_data, axis=1)

In [22]:
class KerasModelCreator:
    def normalise_X(self, X, p=1):
        upper = np.percentile(X, 100-p)
        lower = np.percentile(X, p)
    
        bounded_X = np.where(X > upper, np.median(X), X)
        bounded_X = np.where(X < lower, np.median(X), bounded_X)
        
        scaled_X = (bounded_X - lower)/(upper - lower)
        return scaled_X

    def run(self, X, y, model_path, epochs, overwrite=False):
        ''' 
        If not overwrite and there's an existing model, the model will 
        continue trainingw if the given epoch is bigger than the previous,
        else just evaluate.
        Ensure train splits are the same across continuations / evaluations
        by not modifying the random_state in split_and_normalise.
        '''
        model_savepoint = model_path.parent.joinpath(model_path.stem)
        log_file = model_path.with_suffix('.log')

        if overwrite:
            for f in [model_path, log_file] + list(model_savepoint.glob('*')):
                f.unlink(missing_ok=True)

        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.9, random_state=42)

        for i in range(X.shape[-1]):
            X_train[...,i] = self.normalise_X(X_train[...,i])
            X_val[...,i] = self.normalise_X(X_val[...,i])
        
        default_metrics = ['accuracy', 'root_mean_squared_error', 'r2_score']
        
        if model_path.is_file():
            model = tf.keras.models.load_model(model_path)
        else:
            model = self.build_model(y, X.shape, default_metrics)

        callbacks = [
            tf.keras.callbacks.BackupAndRestore(
                model_savepoint, save_freq='epoch', delete_checkpoint=False
            ),
            tf.keras.callbacks.CSVLogger(
                log_file, append=True
            )
        ]

        if log_file.is_file():
            df = pd.read_csv(log_file)[['epoch', 'loss'] + default_metrics]
            df['epoch'] += 1
            print('Previous training:')
                
            display(HTML(df.to_html(index=False)))
        
        model.fit(
            X_train, y_train, epochs=epochs, verbose=1, batch_size=4, callbacks=callbacks, 
            shuffle=False, #validation_data=(X_val, y_val)
        )

        model.save(model_path)

        print(model.evaluate(X_val, y_val, verbose=0))
        
    def build_model(self, y_train, input_shape, metrics):
        m = tf.keras.Sequential()

        m.add(Input(input_shape[1:]))

        m.add(Conv3D(
            filters=256, kernel_size=2, 
            padding='same', activation='relu'
        ))
        m.add(BatchNormalization())
        m.add(Dropout(0.2))

        m.add(Conv3D(
            filters=512, kernel_size=3, 
            padding='same', activation='relu'
        ))
        m.add(BatchNormalization())
        m.add(Dropout(0.2))

        m.add(GlobalAveragePooling3D())
        # m.add(Flatten())
        
        m.add(Dense(
            y_train.shape[1], 
            activation='softmax', 
        ))
        m.compile(optimizer='rmsprop', loss='mse', metrics=metrics)
        
        return m

In [23]:
train_years = [2017, 2018, 2019]
test_years = [2020]
all_seasons = [9]

model_name = f'conv_seasons_{"_".join(map(str, all_seasons))}_years_{"_".join(map(str, train_years))}.keras'
model_path = model_dir.joinpath(model_name)

train_features = get_features(train_years, all_seasons, data_dir)

In [24]:
KerasModelCreator().run(
    train_features, labels,
    model_path, epochs=3, overwrite=True
)

Epoch 1/3
1260/1260 ━━━━━━━━━━━━━━━━━━━━ 154s 121ms/step - accuracy: 0.3302 - loss: 0.0464 - r2_score: -3.3010 - root_mean_squared_error: 0.2151
Epoch 2/3
1260/1260 ━━━━━━━━━━━━━━━━━━━━ 154s 122ms/step - accuracy: 0.4466 - loss: 0.0375 - r2_score: -1.4171 - root_mean_squared_error: 0.1937
Epoch 3/3
1260/1260 ━━━━━━━━━━━━━━━━━━━━ 154s 122ms/step - accuracy: 0.4855 - loss: 0.0356 - r2_score: -1.4782 - root_mean_squared_error: 0.1887
[0.0377873033285141, 0.48973381519317627, 0.19438956677913666, 0.16203387081623077]


In [25]:
test_features = get_features(test_years, all_seasons, data_dir)
model = tf.keras.models.load_model(model_path)
model.evaluate(test_features, labels, verbose=0, return_dict=True)

{'accuracy': 0.14078719913959503,
 'loss': 0.1033751517534256,
 'r2_score': -0.6789924502372742,
 'root_mean_squared_error': 0.321520060300827}

In [21]:
tf.keras.models.load_model(model_path).summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_3 (Conv3D)               │ (None, 3, 6, 6, 16)    │         1,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 3, 6, 6, 16)    │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 3, 6, 6, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_4 (Conv3D)               │ (None, 3, 6, 6, 32)    │        13,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 3, 6, 6, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 3, 6, 6, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling3d_1      │ (None, 32)             │             0 │
│ (GlobalAveragePooling3D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,352 (126.38 KB)

 Trainable params: 16,127 (63.00 KB)

 Non-trainable params: 96 (384.00 B)

 Optimizer params: 16,129 (63.01 KB)

In [11]:
# import subprocess
# subprocess.run(['sudo', 'shutdown', 'now'])

In [12]:
# season_combinations = itertools.chain.from_iterable(
#     itertools.combinations(all_seasons, r) for r in range(1, len(all_seasons)+1))
# season_combinations = list(season_combinations)